In [1]:
from functools import reduce
import random
import pandas as pd
import names

In [2]:
gate_columns = [
    "Gate ID",
    "Terminal Name",
    "Gate Number",
    "Status",
    "Airport ID"
]

airport_columns = [
    "Airport ID",
    "Name",
    "City"
]

weather_columns = [
    "Weather ID",
    "Conditions",
    "Update Time",
    "Airport ID"
]

aircraft_columns = [
    "Aircraft ID",
    "Aircraft Type",
    "Current Status"
]

passenger_columns = [
    "Passenger ID",
    "Name",
    "Ticket Number",
    "Flight Number"
]

crew_columns = [
    "Crew ID",
    "Crew Type",
    "Member Name",
    "Flight Number"
]

flight_columns = [
    "Flight Number",
    "Departure Time",
    "Arrival Time",
    "Flight Status",
    "Aircraft ID",
    "Departure Gate ID",
    "Departure Airport ID",
    "Arrival Gate ID",
    "Arrival Airport ID"
]

In [4]:
flights = pd.DataFrame(columns=flight_columns)

In [ ]:
def generate_departure_and_arrival_time(seed: int):
    

In [ ]:
for i in range(50000):
    